In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from cassandra.cluster import Cluster
from datetime import datetime


In [2]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()
session.set_keyspace('data_stock')

In [3]:
rows_fin = session.execute("""
    SELECT symbol, eps, pe, pbv, percentyield FROM stock_fun_data ALLOW FILTERING
""")
df_financial = pd.DataFrame(rows_fin, columns=['symbol', 'eps', 'pe', 'pbv', 'percentyield'])
df_financial = df_financial.dropna()

In [4]:
def classify_financial_level(row):
    score = 0
    if row['eps'] > 0:
        score += 1
    if row['pe'] > 0:
        score += 1
    if row['pbv'] < 2:
        score += 1
    if row['percentyield'] > 3:
        score += 1

    if score == 4:
        return 'A'
    elif score == 3:
        return 'B'
    elif score == 2:
        return 'C'
    elif score == 1:
        return 'D'
    else:
        return 'E'

df_financial['financial_level'] = df_financial.apply(classify_financial_level, axis=1)

In [5]:
rows_price = session.execute("""
    SELECT symbol, time, close_price FROM candlestick_data ALLOW FILTERING
""")
df_price = pd.DataFrame(rows_price, columns=['symbol', 'time', 'close_price'])
df_price['time'] = pd.to_datetime(df_price['time'])
df_price = df_price.sort_values(['symbol', 'time'])

In [6]:
def calculate_trend_level(symbol_df):
    if len(symbol_df) < 2:
        return 'c'
    start_price = symbol_df['close_price'].iloc[0]
    end_price = symbol_df['close_price'].iloc[-1]
    change_pct = ((end_price - start_price) / start_price) * 100

    if change_pct > 20:
        return 'a'
    elif change_pct > 10:
        return 'b'
    elif change_pct >= -5:
        return 'c'
    elif change_pct >= -15:
        return 'd'
    else:
        return 'e'

trend_levels = []
for symbol in df_price['symbol'].unique():
    symbol_df = df_price[df_price['symbol'] == symbol]
    level = calculate_trend_level(symbol_df)
    trend_levels.append({'symbol': symbol, 'trend_level': level})

df_trend = pd.DataFrame(trend_levels)

In [7]:
df_quadrant = pd.merge(df_financial, df_trend, on='symbol', how='inner')
df_quadrant['quadrant'] = df_quadrant['financial_level'] + df_quadrant['trend_level']

In [8]:
quadrant_groups = df_quadrant.groupby('quadrant')['symbol'].apply(list).to_dict()
financial_levels = ['A', 'B', 'C', 'D', 'E']
trend_levels = ['a', 'b', 'c', 'd', 'e']


In [9]:
plot_table = []
for f in financial_levels:
    row = []
    for t in trend_levels:
        code = f + t
        stocks = quadrant_groups.get(code, [])
        text = f"<b>{code}</b><br>" + "<br>".join(stocks) if stocks else f"<b>{code}</b><br>-"
        row.append(text)
    plot_table.append(row)

In [10]:
colors = [[{'A': '#2ecc71', 'B': '#58d68d', 'C': '#f4d03f', 'D': '#f39c12', 'E': '#e74c3c'}[f]]*5 for f in financial_levels]

fig = go.Figure(data=go.Table(
    header=dict(
        values=["Super Bullish", "Up Trend", "Sideway", "Down Trend", "Crash"],
        fill_color="#dcdcdc",
        align="center",
        font=dict(color="black", size=14)
    ),
    cells=dict(
        values=plot_table,
        fill_color=colors,
        align="center",
        height=100,
        font=dict(color="white", size=12)
    )
))

fig.update_layout(title="📊 Stock Quadrant จาก Cassandra")
fig.show()
